# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [ ]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


In [ ]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms, \
    compute_affine_transformation, compute_affine_transformation_centroid
from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [ ]:
def absolute_sum(l):
    la = np.array(l)
    nabs = (np.array(la*la))
    return np.sum(la, axis=0)

def sum_square_com(com):
    ss = np.sqrt(sum([s*s for s in com]))
    return ss

def generate_combinations(lst):
    """
    Generate all combinations of at least 3 elements from the given list.
    
    :param lst: List of elements
    :return: List of tuples containing the combinations
    """
    return list(combinations(lst, 5 ))

    #result = []
    #for r in range(3, len(lst) + 1):
    #    result.extend(combinations(lst, r))
    #return result

def get_umeyama(atlas_src, allen_src):

    A, t = umeyama(atlas_src.T, allen_src.T, with_scaling=True)
    transformation_matrix = np.hstack( [A, t ])
    transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])

    return transformation_matrix

In [ ]:
atlas_all = list_coms('Atlas')
allen_all = list_coms('Allen')
common_keys = list(atlas_all.keys() & allen_all.keys())
atlas_common = np.array([atlas_all[s] for s in common_keys])
allen_common = np.array([allen_all[s] for s in common_keys])
print(len(common_keys))

In [ ]:
base_keys = ['LRt_L', 'LRt_R', 'SC', 'SNC_R', 'SNC_L']
test_keys = ['PBG_L', 'PBG_R', 'SC', 'IC', '4N_L', '4N_R', 'LRt_L','LRt_R', 'SNC_L', 'SNC_R', ]
bad_keys = ['RtTg','AP']
good_keys = set(common_keys) - set(bad_keys)
print(len(base_keys))
print(len(test_keys))
print(len(bad_keys))
print(len(good_keys))
print(len(common_keys))

In [ ]:
atlas_src = np.array([atlas_all[s] for s in base_keys])
allen_src = np.array([allen_all[s] for s in base_keys])
matrix = compute_affine_transformation(atlas_src, allen_src)

In [ ]:
df_list = []
error = []
for structure in common_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = np.array(allen_all[structure]) 
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,4) for x in transformed]
    difference = [round(a - b,4) for a, b in zip(transformed, allen0)]
    ss = sum_square_com(difference)
    row = [structure, atlas0, allen0, transformed, difference, ss]
    df_list.append(row)
    error.append(ss)
print('RMS', sum(error)/len(df_list))
# test keys RMS 25.61709479307028
# base keys RMS 25.602246865972344

In [ ]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
HTML(df.to_html(index=False))

In [ ]:
df['sumsquares'].sum()/len(df.index)

In [ ]:
949.4215777978649/len(df.index)

In [ ]:
df_list = []
atlas_src = np.array([atlas_all[s] for s in remaining_keys])
for structure in remaining_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = 0
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,2) for x in transformed]
    difference = 0
    ss = 0
    row = [structure, atlas0, allen0, transformed, difference, ss]
    df_list.append(row)

In [ ]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
HTML(df.to_html(index=False))